In [1]:
import pandas as pd
import re
from datetime import datetime as dt
import pickle

In [2]:
exits=pd.read_csv("../output/raw_exits.csv")
funds=pd.read_csv("../output/raw_funding.csv")

In [3]:
exits.name = exits

In [5]:
exits.shape

(1670, 37)

In [4]:
funds.shape

(9523, 10)

In [7]:
funds.sort_values("Organization Name")

Announced Date Equity Only Funding        Funding Stage  \
18     Nov 21, 2017                 Yes  Early Stage Venture   
6035   Oct 13, 2015                 Yes  Early Stage Venture   
8568   Oct 24, 2017                 Yes  Early Stage Venture   
5777   Oct 24, 2017                 Yes  Early Stage Venture   
7845   Oct 13, 2015                 Yes  Early Stage Venture   
27     Oct 25, 2017                 Yes                 Seed   
8939   Aug 23, 2016                 Yes                 Seed   
8658    Jul 1, 2017                 Yes                 Seed   
8314   Feb 16, 2011                 Yes  Early Stage Venture   
6256   Jul 31, 2014                 Yes  Early Stage Venture   
8287   Apr 28, 2011                 Yes  Early Stage Venture   
604    Oct 20, 2015                 Yes                 Seed   
6116   Apr 20, 2015                 Yes  Early Stage Venture   
5822   Jun 30, 2017                 Yes                  NaN   
5972   Apr 20, 2016                 Yes  Early Stage Venture   
5815   Jul 26, 2017                 Yes   Late Stage Venture   
8102    Oct 1, 2012                 Yes                 Seed   
9344    Dec 1, 2014                 Yes                 Seed   
9244    May 1, 2015                 Yes                 Seed   
8424    Oct 1, 2009                 Yes  Early Stage Venture   
6671    Oct 1, 2010                 Yes  Early Stage Venture   
4698    Oct 3, 2007                 Yes  Early Stage Venture   
4321   Dec 21, 2012                 Yes   Late Stage Venture   
4049    Jul 3, 2015                 Yes   Late Stage Venture   
5795    Sep 5, 2017                 Yes   Late Stage Venture   
4512    Nov 9, 2010                 Yes   Late Stage Venture   
4591   Jun 18, 2009                 Yes  Early Stage Venture   
7156    Jul 6, 2016                  No                  NaN   
7112   Aug 22, 2016                 Yes                 Seed   
6877   Apr 27, 2017                  No                  NaN   
...             ...                 ...                  ...   
5643   Jan 13, 1998                  No                  NaN   
985    Apr 18, 2014                 Yes                 Seed   
561    Nov 24, 2015                 Yes                 Seed   
5788   Sep 19, 2017                 Yes  Early Stage Venture   
158    Mar 20, 2017                 Yes  Early Stage Venture   
992    Apr 14, 2014                 Yes  Early Stage Venture   
838     Nov 1, 2014                 Yes                 Seed   
7795   Oct 12, 2016                 Yes  Early Stage Venture   
534    Dec 27, 2015                 Yes                 Seed   
1756    Jan 2, 2002                 Yes   Late Stage Venture   
3358    Dec 9, 2009                 Yes  Early Stage Venture   
2891   Mar 25, 2015                 Yes                  NaN   
3467    Dec 1, 2007                 Yes  Early Stage Venture   
3109   Oct 18, 2012                 Yes   Late Stage Venture   
3273   Feb 24, 2011                 Yes   Late Stage Venture   
9429   Jul 16, 2014                 Yes                 Seed   
3086    Mar 7, 2013                 Yes   Late Stage Venture   
3101   Dec 20, 2012                 Yes   Late Stage Venture   
758    Apr 17, 2015                 Yes                  NaN   
6813   Sep 12, 2017                 Yes                 Seed   
3175    Mar 1, 2012                 Yes                 Seed   
445     Mar 1, 2016                 Yes                  NaN   
794     Feb 9, 2015                 Yes  Early Stage Venture   
6661    Nov 1, 2010                 Yes                  NaN   
6605    Apr 1, 2011                 Yes  Early Stage Venture   
6630    Jan 1, 2011                 Yes  Early Stage Venture   
8091   Nov 14, 2012                 Yes                 Seed   
8702   Jun 26, 2017                 Yes                 Seed   
9167    Sep 1, 2015                 Yes                 Seed   
9144    Sep 1, 2015                 Yes                 Seed   

                  Funding Type      

<h2>Remove duplicate investments by multiple VCs</h2> 

In [10]:
f_uq = funds[funds.duplicated(subset=["Funding Type", "Organization Name"], keep = "last") == False]

In [11]:
f_uq.shape

(8078, 10)

In [12]:
f_uq["Funding Type"].value_counts()

Seed                        2514
Series A                    1392
Series B                    1155
Series C                     746
Venture - Series Unknown     614
Grant                        587
Series D                     364
Convertible Note             262
Series E                     154
Angel                         95
Series F                      64
Private Equity                40
Debt Financing                28
Funding Round                 20
Series G                      18
Post-IPO Equity               13
Secondary Market               4
Corporate Round                3
Series H                       2
Initial Coin Offering          1
Equity Crowdfunding            1
Product Crowdfunding           1
Name: Funding Type, dtype: int64

In [11]:
f_uq.groupby(["Organization Name", "Funding Type"])["Money Raised"].apply(sum)

Organization Name                       Funding Type            
 Justmop.com                            Series A                             NaN
 NimbleRx                               Series A                     $12,000,000
                                        Series B                     $28,000,000
 SpotDraft                              Seed                            $550,000
/dev/color                              Seed                            $120,000
10 By 10                                Seed                            $120,000
1000memories                            Series A                      $2,520,000
1006.tv                                 Series B                     $10,000,000
140 Proof                               Series B                      $2,500,000
1Huddle                                 Seed                            $125,000
1mg                                     Series A                      $6,000,000
                                        Seri

<h3> Use only Seed --> Series C data </h3>

In [12]:
f_rounds = f_uq[(f_uq["Funding Type"] == "Seed") |
                (f_uq["Funding Type"] == "Series A") |
                (f_uq["Funding Type"] == "Series B") |
                (f_uq["Funding Type"] == "Series C")]

In [13]:
f_rounds["Funding Type"].value_counts()

Seed        2514
Series A    1392
Series B    1155
Series C     746
Name: Funding Type, dtype: int64

In [14]:
f_date = f_rounds.pivot(index = "Organization Name" , columns = "Funding Type", values = "Announced Date")

# df.rename(columns=lambda x: x.replace('$', ''), inplace=True)

f_date.rename(columns = {"Seed": "Seed_date",
               "Series A": "A_date",
               "Series B": "B_date",
               "Series C": "C_date"}, inplace = True)

In [15]:
f_money = f_rounds.pivot(index = "Organization Name", columns = "Funding Type", values = "Money Raised")

In [16]:
f_trans = pd.concat([f_money, f_date], axis = 1, join = 'outer', ignore_index = False)
f_trans

Funding Type                                  Seed     Series A     Series B  \
Organization Name                                                              
 Justmop.com                                  None          NaN         None   
 NimbleRx                                     None  $12,000,000  $28,000,000   
 SpotDraft                                $550,000         None         None   
/dev/color                                $120,000         None         None   
10 By 10                                  $120,000         None         None   
1000memories                                  None   $2,520,000         None   
1006.tv                                       None         None  $10,000,000   
140 Proof                                     None         None   $2,500,000   
1Huddle                                   $125,000         None         None   
1mg                                           None   $6,000,000  $16,000,000   
1stdibs                                        NaN         None         None   
20n                                       $120,000         None         None   
20x200                                        None     $800,000         None   
2345.com                                      None         None          NaN   
23andMe                                       None   $8,953,320  $13,600,000   
247tickets                                     NaN         None         None   
2Wire                                         None         None         None   
3-V Biosciences                               None         None  $30,000,000   
37coins                                   $500,000         None         None   
3Jam                                          None   $4,000,000         None   
3LM                                     $1,500,000         None         None   
3TEN8.AI                                $1,670,000         None         None   
3VR                                           None         None  $10,000,000   
41st Parameter                                None         None  $11,200,000   
42 Technologies                                NaN         None         None   
42Floors                                  $400,000         None  $12,000,000   
43Layers                                       NaN         None         None   
4INFO                                         None         None         None   
4th Paradigm                                  None          NaN         None   
4tigo                                         None  $10,000,000         None   
...                                            ...          ...          ...   
netvmg                                        None  $10,000,000  $57,000,000   
newBrandAnalytics                             None         None  $26,000,000   
ngmoco                                        None   $5,600,000  $10,000,000   
nprogress                               $1,700,000         None         None   
nurseVersity                              $125,000         None         None   
ooma                                          None   $8,000,000  $18,000,000   
payleven                                      None   €2,000,000  €12,500,000   
quixi                                         None  $27,500,000         None   
rakam                                     $150,000         None         None   
resin.io                                      None   $3,000,000         None   
simplifund                                $125,000         None         None   
so-sure                                   $150,000         None         None   
sourceeasy                                $126,500         None         None   
sp0n                                          None  $12,000,000         None   
springcleaning.ae                             None          NaN         None   
storefront                                    None   $7,300,000         None   
strapping                                 $100,000         None         None   


In [17]:
f_trans["index"] = f_trans.index
print(f_trans.shape)
print(f_trans["index"].nunique())

(4471, 9)
4471


In [18]:
f_trans[f_trans.duplicated(keep = False)].sort_values("index")

Empty DataFrame
Columns: [Seed, Series A, Series B, Series C, Seed_date, A_date, B_date, C_date, index]
Index: []

In [19]:
f_trans[f_trans.index == "Agami System"]

Funding Type       Seed    Series A     Series B     Series C Seed_date  \
Organization Name                                                         
Agami System       None  $5,500,000  $24,000,000  $50,000,000      None   

Funding Type            A_date        B_date        C_date         index  
Organization Name                                                         
Agami System       Jun 1, 2004  Sep 20, 2004  Nov 21, 2007  Agami System

In [20]:
# col = f_trans.columns.values

# for col in f_trans:
#     print(f_trans[col].notnull().value_counts())

In [21]:
f_trans.dropna(axis = 0, how = "any", thresh = 5)

Funding Type                        Seed      Series A      Series B  \
Organization Name                                                      
 NimbleRx                           None   $12,000,000   $28,000,000   
1mg                                 None    $6,000,000   $16,000,000   
23andMe                             None    $8,953,320   $13,600,000   
3-V Biosciences                     None          None   $30,000,000   
3VR                                 None          None   $10,000,000   
41st Parameter                      None          None   $11,200,000   
42Floors                        $400,000          None   $12,000,000   
99.co                         $1,600,000    $7,900,000          None   
99designs                           None   $35,000,000   $10,000,000   
AOptix Technologies                 None    $5,700,000   $18,000,000   
ARMO BioSciences                    None   $20,000,000   $30,000,000   
Abe's Market                        None    $3,400,000    $5,000,000   
Acopia Networks                     None   $10,960,000   $30,000,000   
ActionIQ                      $2,000,000   $13,000,000   $30,000,000   
AdMob                               None          None   $15,000,000   
AdRoll                              None    $4,000,000   $15,000,000   
AdXpose                             None          None   $10,000,000   
Adallom                             None    $4,500,000   $15,000,000   
AddThis                             None          None   $10,000,000   
Aera Technology                     None          None   $12,000,000   
Aerospike                           None          None    $8,000,000   
Agami System                        None    $5,500,000   $24,000,000   
Aggregate Knowledge                 None    $5,000,000   $20,000,000   
AgilOne                             None    $6,000,000   $10,000,000   
Agilyx                              None          None   $22,000,000   
Air Works India Engineering         None          None   $27,000,000   
Airbnb                              None    $7,200,000  $112,000,000   
Airgo Networks                      None          None   $20,000,000   
Airware                             None          None   $25,000,000   
Alfresco                            None    $2,500,000    $8,000,000   
...                                  ...           ...           ...   
Zipline                             None    $5,000,000   $25,000,000   
ZoomCar                             None    $8,000,000   $24,000,000   
Zoomdata                      $1,100,000    $4,099,999   $17,000,000   
Zoox                         $40,000,000  $250,000,000          None   
Zosano Pharma                       None   $90,000,000   $30,000,000   
Zumper                        $1,000,000    $6,500,000   $17,600,000   
Zuoyebang                           None   $25,000,000   $60,000,000   
Zylo                          $3,300,000    $9,300,000          None   
Zymergen                            None   $42,140,000  $130,000,000   
Zynga                               None   $10,000,000   $25,000,000   
appOrbit                      $1,000,000   $10,000,000          None   
d.light design                      None    $6,000,000          None   
flaregames                          None    €6,000,000   $12,200,000   
homelane                            None    $4,500,000   $50,000,000   
iControl Networks                   None          None   $15,500,000   
iPIN                                None          None   $23,000,000   
imgix                           $164,000    $5,000,000          None   
iolon, inc.                         None          None   $53,000,000   
isocket                       $2,104,119    $8,000,000          None   
letgo                               None          None  $100,000,000   
lynda.com                           None  $103,000,000  $186,000,000   
migme                                NaN   $10,000,000   $13,500,000   
mo9 (moKredit)                      None

In [22]:
f_trans

Funding Type                                  Seed     Series A     Series B  \
Organization Name                                                              
 Justmop.com                                  None          NaN         None   
 NimbleRx                                     None  $12,000,000  $28,000,000   
 SpotDraft                                $550,000         None         None   
/dev/color                                $120,000         None         None   
10 By 10                                  $120,000         None         None   
1000memories                                  None   $2,520,000         None   
1006.tv                                       None         None  $10,000,000   
140 Proof                                     None         None   $2,500,000   
1Huddle                                   $125,000         None         None   
1mg                                           None   $6,000,000  $16,000,000   
1stdibs                                        NaN         None         None   
20n                                       $120,000         None         None   
20x200                                        None     $800,000         None   
2345.com                                      None         None          NaN   
23andMe                                       None   $8,953,320  $13,600,000   
247tickets                                     NaN         None         None   
2Wire                                         None         None         None   
3-V Biosciences                               None         None  $30,000,000   
37coins                                   $500,000         None         None   
3Jam                                          None   $4,000,000         None   
3LM                                     $1,500,000         None         None   
3TEN8.AI                                $1,670,000         None         None   
3VR                                           None         None  $10,000,000   
41st Parameter                                None         None  $11,200,000   
42 Technologies                                NaN         None         None   
42Floors                                  $400,000         None  $12,000,000   
43Layers                                       NaN         None         None   
4INFO                                         None         None         None   
4th Paradigm                                  None          NaN         None   
4tigo                                         None  $10,000,000         None   
...                                            ...          ...          ...   
netvmg                                        None  $10,000,000  $57,000,000   
newBrandAnalytics                             None         None  $26,000,000   
ngmoco                                        None   $5,600,000  $10,000,000   
nprogress                               $1,700,000         None         None   
nurseVersity                              $125,000         None         None   
ooma                                          None   $8,000,000  $18,000,000   
payleven                                      None   €2,000,000  €12,500,000   
quixi                                         None  $27,500,000         None   
rakam                                     $150,000         None         None   
resin.io                                      None   $3,000,000         None   
simplifund                                $125,000         None         None   
so-sure                                   $150,000         None         None   
sourceeasy                                $126,500         None         None   
sp0n                                          None  $12,000,000         None   
springcleaning.ae                             None          NaN         None   
storefront                                    None   $7,300,000         None   
strapping                                 $100,000         None         None   


In [23]:
print(exits.columns)
e_reduce = exits[["Organization Name", "Categories", "Category Groups", "Founded Date", 
                  "Funding Status", "Operating Status", "Total Funding Amount", 
                   "Total Equity Funding Amount"]]

e_reduce

Index(['Acquisition Status', 'CB Rank (Organization)', 'Categories',
       'Category Groups', 'Closed Date', 'Company Type', 'Delisted Date',
       'Description', 'Founded Date', 'Funding Status',
       'Headquarters Location', 'IPO Date', 'IPO Status', 'Investment Stage',
       'Investor', 'Investor Type', 'Last Equity Funding Amount',
       'Last Equity Funding Type', 'Last Funding Amount', 'Last Funding Date',
       'Last Funding Type', 'Money Raised at IPO', 'Number of Employees',
       'Number of Exits', 'Number of Founders', 'Number of Funding Rounds',
       'Number of Investments', 'Number of Investors',
       'Number of Lead Investments', 'Number of Lead Investors',
       'Number of Portfolio Organizations', 'Operating Status',
       'Organization Name', 'Organization Name URL',
       'Total Equity Funding Amount', 'Total Funding Amount',
       'Valuation at IPO'],
      dtype='object')


Organization Name  \
0                               Reddit   
1                             SendGrid   
2                              Sunrise   
3                               Simple   
4           Animoca Brands Corporation   
5                               Twilio   
6                             Visually   
7                                 ZOZI   
8                           SkyGiraffe   
9                           Rapportive   
10                             Keen IO   
11               Valor Water Analytics   
12                                Viki   
13                           FabFitFun   
14    LookSharp (powering InternMatch)   
15                            Wildfire   
16                             ToutApp   
17                              Virool   
18                     Creative Market   
19                           Medialets   
20                              Venzee   
21                          ChangeCoin   
22                           Boatbound   
23                            Farmeron   
24                             Clarity   
25                                Gyft   
26                             Behance   
27                          AdEspresso   
28                            MakerBot   
29                             Apsalar   
...                                ...   
1640                         280 North   
1641                          GazeHawk   
1642                         TextPayMe   
1643                       Munch On Me   
1644                           Parakey   
1645                         Simperium   
1646                           Snapjoy   
1647                             Lingt   
1648                            reMail   
1649                        Posmetrics   
1650                       Hackermeter   
1651                          Yardsale   
1652                        Threadable   
1653                       Anywhere.FM   
1654                           Knowmia   
1655                          Voicegem   
1656                           Fanvibe   
1657                           Sendoid   
1658                          Glassmap   
1659                         TapEngage   
1660                          FathomDB   
1661                            gantto   
1662                            Heysan   
1663                 Thought Mechanics   
1664                           Echodio   
1665                          Snipshot   
1666                         Clutch.io   
1667                            Zenter   
1668                      Front Studio   
1669                        Teleborder   

                                             Categories  \
0     Content, News, Social Bookmarking, Social Medi...   
1     Analytics, Business Information Systems, Email...   
2                           Apps, Email, Events, Mobile   
3     Banking, Enterprise Software, Financial Servic...   
4                       Developer Tools, Gaming, Mobile   
5              Enterprise Software, Internet, SMS, VoIP   
6     Brand Marketing, Content, Internet, Presentations   
7                     E-Commerce, SaaS, Tourism, Travel   
8                       Android, CRM, iOS, Mobile, SaaS   
9                                Email, Messaging, SaaS   
10    Analytics, Big Data, Cloud Computing, Develope...   
11               Analytics, Big Data, Natural Resources   
12                    Music, TV, Video, Video Streaming   
13    Digital Entertainment, Digital Media, E-Commer...   
14    Career Planning, Education, Employment, Social...   
15       Advertising, Enterprise Software, Social Media   
16    Email, Enterprise Software, Internet, Sales Au...   
17            Advertising, Advertising Platforms, Video   
18                 E-Commerce Platforms, Graphic Design   
19    Ad Server, Advertising, Mobile, Mobile Adverti...   
20    E-Commerce, Manufacturing, Supply Chain Manage...   
21                       Bitcoin, Payments, Web Hosting   
22      Boating, Collaborative Consumption, Marketplace   


In [24]:
print(e_reduce.shape)
print(e_reduce["Organization Name"].nunique())

e_reduce = e_reduce.drop_duplicates(["Organization Name"], keep = 'first')
result = e_reduce.shape[0] == e_reduce["Organization Name"].nunique()
print("Drop successful? {}".format(result))

(1670, 8)
1334
Drop successful? True


In [25]:
e_dict = e_reduce.set_index("Organization Name")[['Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount']]

e_dict.shape

(1334, 7)

<h2>Merge the funding rounds with startup info</h2>

In [278]:
f_trans_all = f_trans.merge(e_dict, how = 'outer', left_index = True, right_index = True)
f_trans_all.shape

(5006, 16)

In [279]:
f_trans_all

Seed     Series A     Series B  \
Organization Name                                                              
 Justmop.com                                  None          NaN         None   
 NimbleRx                                     None  $12,000,000  $28,000,000   
 SpotDraft                                $550,000         None         None   
/dev/color                                $120,000         None         None   
10 By 10                                  $120,000         None         None   
1000memories                                  None   $2,520,000         None   
1006.tv                                       None         None  $10,000,000   
140 Proof                                     None         None   $2,500,000   
1Huddle                                   $125,000         None         None   
1mg                                           None   $6,000,000  $16,000,000   
1stdibs                                        NaN         None         None   
20n                                       $120,000         None         None   
20x200                                        None     $800,000         None   
2345.com                                      None         None          NaN   
23andMe                                       None   $8,953,320  $13,600,000   
247tickets                                     NaN         None         None   
280 North                                      NaN          NaN          NaN   
2Wire                                         None         None         None   
3-V Biosciences                               None         None  $30,000,000   
37coins                                   $500,000         None         None   
3Jam                                          None   $4,000,000         None   
3LM                                     $1,500,000         None         None   
3TEN8.AI                                $1,670,000         None         None   
3VR                                           None         None  $10,000,000   
3ware                                          NaN          NaN          NaN   
41st Parameter                                None         None  $11,200,000   
42 Technologies                                NaN         None         None   
42Floors                                  $400,000         None  $12,000,000   
43Layers                                       NaN         None         None   
4INFO                                         None         None         None   
...                                            ...          ...          ...   
payleven                                      None   €2,000,000  €12,500,000   
play140                                        NaN          NaN          NaN   
quixi                                         None  $27,500,000         None   
rakam                                     $150,000         None         None   
reMail                                         NaN          NaN          NaN   
resin.io                                      None   $3,000,000         None   
seeUthere.com                                  NaN          NaN          NaN   
simplifund                                $125,000         None         None   
slinkset                                       NaN          NaN          NaN   
so-sure                                   $150,000         None         None   
sourceeasy                                $126,500         None         None   
sp0n                                          None  $12,000,000         None   
springcleaning.ae                             None          NaN         None   
storefront                                    None   $7,300,000         None   
strapping                                 $100,000         None         None   
tCell                                         None   $9,400,000         None   
tagMonkey                                 $125,000         None         None   
terraspring                                   

In [280]:
print(f_trans_all.dropna(axis = 0, how = "any", thresh = 11).shape)
f_trans_all.dropna(axis = 0, how = "any", thresh = 11)

(267, 16)


Seed      Series A      Series B      Series C  \
Organization Name                                                              
3VR                           None          None   $10,000,000    $9,000,000   
41st Parameter                None          None   $11,200,000   $10,070,000   
ARMO BioSciences              None   $20,000,000   $30,000,000   $67,000,000   
Abe's Market                  None    $3,400,000    $5,000,000          None   
Acopia Networks               None   $10,960,000   $30,000,000   $25,000,000   
AdMob                         None          None   $15,000,000   $12,500,000   
AdXpose                       None          None   $10,000,000    $3,000,000   
AddThis                       None          None   $10,000,000   $18,000,000   
Aggregate Knowledge           None    $5,000,000   $20,000,000    $9,000,000   
Airgo Networks                None          None   $20,000,000   $20,000,000   
Alfresco                      None    $2,500,000    $8,000,000   $13,000,000   
Altiscale                     None   $12,000,000   $30,000,000          None   
Altor Networks                None    $6,000,000   $10,000,000          None   
Amobee                        None    $5,000,000   $15,000,000   $22,000,000   
Amyris Biotechnologies        None   $20,000,000   $70,000,000   $41,750,000   
Anagran                       None    $8,000,000   $14,000,000   $12,000,000   
Apcera                  $2,200,000    $5,000,000          None          None   
Apperian                      None          None   $12,400,000   $12,000,000   
Arcot Systems                 None   $11,000,000   $23,000,000          None   
Asera                         None          None   $47,400,000  $115,000,000   
Atrica                        None   $16,000,000   $18,000,000          None   
Ausra                         None   $43,300,000   $24,500,000          None   
Autoquake                     None    $6,000,000   $11,890,000          None   
Baby.com.br                   None    $4,400,000   $16,700,000          None   
Baidu                         None          None   $10,000,000   $15,000,000   
Big Bear Networks             None   $20,000,000   $40,000,000   $18,000,000   
BinOptics                     None    $1,731,000   $10,000,000    $6,000,000   
Blue Nile                     None          None   $42,000,000    $7,000,000   
Bluebox                       None    $9,500,000   $18,000,000          None   
BodyMedia                     None          None    $6,500,000    $4,000,000   
...                            ...           ...           ...           ...   
Vertica Systems               None    $7,000,000   $16,500,000          None   
ViaFone                       None   $10,700,000   $23,000,000          None   
Virool                  $6,620,000   $12,000,000          None          None   
Visible Path                  None    $7,700,000   $17,000,000          None   
Vivotech                      None          None   $16,239,999   $22,500,000   
Vuclip                        None    $8,100,000    $6,000,000    $8,000,000   
Waze                          None   $12,000,000   $25,000,000   $30,000,000   
Wetpaint                      None          None    $9,500,000   $25,000,000   
WiChorus                      None          None   $15,000,000   $18,000,000   
Wily Technology               None          None   $10,000,000   $15,000,000   
Xcellerex                     None          None   $11,000,000   $31,000,000   
XenSource                     None    $6,000,000   $17,000,000   $15,000,000   
Xfire                         None    $1,000,000    $1,500,000    $5,000,000   
Xtime                         None          None    $2,600,000    $7,250,000   
Yodle                         None    $3,000,000   $12,000,000   $10,000,000   
YuMe                           NaN    $7,000,000    $9,000,000    $5,000,000   
ZAI Lab                       None   $30,000,000  $100,000,000          None   
Zarget                  $1,500

## Convert Dates


In [281]:
# x = f_trans_all.loc["99.co", "A_date_dt"] - f_trans_all.loc["99.co", "Seed_date_dt"]
f_trans_all["Seed_date_dt"] = pd.to_datetime(f_trans_all["Seed_date"], errors="ignore", format = "%b %d, %Y")
f_trans_all["A_date_dt"] = pd.to_datetime(f_trans_all["A_date"], errors="ignore", format = "%b %d, %Y")
f_trans_all["Seed_A"] = f_trans_all["A_date_dt"] - f_trans_all["Seed_date_dt"]
f_trans_all.dropna(subset=["Seed_date_dt", "A_date_dt"])[["Seed_date_dt", "A_date_dt", "Seed_A"]]

Seed_date_dt  A_date_dt  \
Organization Name                                                      
99.co                                          2015-01-29 2017-04-06   
ActionIQ                                       2014-08-21 2017-04-03   
Algolia                                        2014-06-19 2015-05-20   
Alodokter                                      2015-04-21 2016-08-05   
Alooma                                         2014-08-18 2016-03-14   
Althea Inc.                                    2015-10-15 2016-05-26   
Amitree                                        2014-01-08 2017-08-25   
Anchor                                         2016-02-09 2017-09-28   
Apcera                                         2012-05-08 2013-07-08   
Area 1 Security                                2014-05-05 2014-12-10   
Ascend.io                                      2015-12-15 2017-05-17   
Attero                                         2008-04-18 2008-08-21   
Ava Winery                                     2016-08-05 2017-12-21   
Away                                           2015-08-17 2016-09-08   
BTCjam                                         2014-06-05 2014-12-24   
BabbaCo (acquired by Barefoot Books in 2014)   2011-08-01 2012-05-01   
Babybe GmbH                                    2013-07-12 2016-08-29   
BaubleBar                                      2010-11-05 2012-07-23   
Birchbox                                       2010-10-27 2011-08-17   
Bitrise                                        2017-01-01 2017-07-31   
Bombfell                                       2013-02-13 2015-01-23   
Bonsai                                         2015-11-21 2016-11-03   
Boom Technology                                2015-11-21 2017-03-22   
Boundless                                      2011-04-28 2012-04-05   
Boxbee, Inc.                                   2014-04-16 2014-12-16   
Branch                                         2012-03-06 2015-02-26   
Brat                                           2017-07-24 2017-11-03   
Bump Technologies                              2009-06-01 2009-10-24   
Bus.com                                        2015-11-21 2017-04-05   
Canopy                                         2014-07-23 2015-06-23   
...                                                   ...        ...   
Transit                                        2016-09-21 2017-09-26   
URX                                            2013-09-01 2014-04-28   
UrbanClap                                      2015-04-16 2015-06-29   
Verbling                                       2012-01-26 2014-12-12   
Veriflow                                       2014-10-17 2016-07-06   
Vidme                                          2015-04-07 2016-12-14   
ViralGains                                     2014-07-31 2016-01-06   
Virool                                         2013-02-14 2016-04-05   
Vungle                                         2012-05-02 2013-08-01   
WaterSmart Software                            2011-05-16 2013-08-27   
WayUp                                          2014-09-18 2015-04-30   
Weave                                          2014-05-01 2014-06-11   
WibiData                                       2010-09-16 2012-02-07   
Willing                                        2015-09-01 2016-11-03   
Yoshi                                          2016-08-23 2017-01-12   
YuMe                                           2006-08-01 2007-03-05   
Zaarly                                         2011-03-09 2011-10-24   
Zarget                                         2016-05-09 2016-11-02   
Zero                                           2016-09-20 2017-10-23   
Zilingo                                        2015-11-09 2016-09-05   
ZipMatch                                       2014-08-14 2015-03-24   
Zoomdata                                       2012-11-13 2013-07-10   
Zoox                                           2015-07-03 2016-10-04   
Zumper       

In [282]:
dates_string = ["Seed_date", "A_date", "B_date", "C_date"]
dates = []
def convert_datetime(dates_string, df):
    for date in dates_string:
        new_col_name = date + "_dt"
        dates.append(new_col_name)
        df[new_col_name] = pd.to_datetime(df[date], errors="ignore", format = "%b %d, %Y")
        
def time_to_next_round(dates, df):
    for i in range(len(dates)-1):
        new_col_name = dates[i][0] + "_to_" + dates[i+1][0]
        
        df[new_col_name] = df[dates[i+1]] - df[dates[i]]

In [283]:
convert_datetime(dates_string, f_trans_all)
time_to_next_round(dates, f_trans_all)

In [284]:
f_trans_all.dropna(subset = [["S_to_A", "A_to_B", "B_to_C"]], thresh = 2)[["index", "Seed_date_dt", "A_date_dt", "B_date_dt", "C_date_dt",
            "S_to_A", "A_to_B", "B_to_C"]]

index Seed_date_dt  A_date_dt  \
Organization Name                                                        
1mg                                        1mg          NaT 2015-04-20   
23andMe                                23andMe          NaT 2007-10-03   
ARMO BioSciences              ARMO BioSciences          NaT 2013-11-25   
Acopia Networks                Acopia Networks          NaT 2002-10-03   
ActionIQ                              ActionIQ   2014-08-21 2017-04-03   
Adallom                                Adallom          NaT 2012-12-01   
Agami System                      Agami System          NaT 2004-06-01   
Aggregate Knowledge        Aggregate Knowledge          NaT 2006-06-01   
AgilOne                                AgilOne          NaT 2011-01-01   
Airbnb                                  Airbnb          NaT 2010-11-10   
Alfresco                              Alfresco          NaT 2005-07-01   
Algolia                                Algolia   2014-06-19 2015-05-20   
Amino                                    Amino          NaT 2014-09-01   
Amobee                                  Amobee          NaT 2006-11-01   
Amyris Biotechnologies  Amyris Biotechnologies          NaT 2006-10-12   
Anagran                                Anagran          NaT 2004-10-14   
Apprenda                              Apprenda          NaT 2009-11-09   
Arrail Dental Clinic      Arrail Dental Clinic          NaT 2010-03-01   
Autonomic Technologies  Autonomic Technologies          NaT 2008-01-14   
Away                                      Away   2015-08-17 2016-09-08   
Barefoot Networks            Barefoot Networks          NaT 2014-05-14   
BaubleBar                            BaubleBar   2010-11-05 2012-07-23   
Beequick                              Beequick          NaT 2014-10-01   
Behalf                                  Behalf          NaT 2013-09-03   
Big Bear Networks            Big Bear Networks          NaT 2000-08-21   
BinOptics                            BinOptics          NaT 2003-02-26   
Birchbox                              Birchbox   2010-10-27 2011-08-17   
Bitglass                              Bitglass          NaT 2013-03-08   
BlueJeans Network            BlueJeans Network          NaT 2009-11-01   
Box                                        Box          NaT 2006-10-01   
...                                        ...          ...        ...   
VIPKID                                  VIPKID          NaT 2014-05-01   
Viptela                                Viptela          NaT 2012-08-28   
Vuclip                                  Vuclip          NaT 2008-05-02   
VytronUS                              VytronUS          NaT 2007-08-20   
WaterSmart Software        WaterSmart Software   2011-05-16 2013-08-27   
WayUp                                    WayUp   2014-09-18 2015-04-30   
Waze                                      Waze          NaT 2008-03-01   
Weave                                    Weave   2014-05-01 2014-06-11   
WibiData                              WibiData   2010-09-16 2012-02-07   
WorldRemit                          WorldRemit          NaT 2014-03-11   
XenSource                            XenSource          NaT 2005-01-13   
Xfire                                    Xfire          NaT 2003-02-06   
Xiu.com                                Xiu.com          NaT 2011-03-01   
Yodle                                    Yodle          NaT 2006-11-01   
YuMe                                      YuMe   2006-08-01 2007-03-05   
ZettaCore                            ZettaCore          NaT 2002-05-14   
Zilingo                                Zilingo   2015-11-09 2016-09-05   
Zinka Logistics                Zinka Logistics          NaT 2015-07-22   
ZoomCar                                ZoomCar          NaT 2014-10-31   
Zoomdata                              Zoomdata   2012-11-13 2013-07-10   
Zumper                                  Zumper   2012-09-27 2014-03-04   
Zuoyebang                            Zuoyeban

In [285]:
f_trans_all.columns

Index(['Seed', 'Series A', 'Series B', 'Series C', 'Seed_date', 'A_date',
       'B_date', 'C_date', 'index', 'Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount', 'Seed_date_dt',
       'A_date_dt', 'Seed_A', 'B_date_dt', 'C_date_dt', 'S_to_A', 'A_to_B',
       'B_to_C'],
      dtype='object')

## Convert Currencies

In [286]:
# for money in f_trans_all["Seed"]:
#     print(money)

In [287]:
f_trans.iloc[8:25, :]

Funding Type             Seed    Series A     Series B     Series C  \
Organization Name                                                     
1Huddle              $125,000        None         None         None   
1mg                      None  $6,000,000  $16,000,000  $15,000,000   
1stdibs                   NaN        None         None         None   
20n                  $120,000        None         None         None   
20x200                   None    $800,000         None         None   
2345.com                 None        None          NaN         None   
23andMe                  None  $8,953,320  $13,600,000  $22,220,289   
247tickets                NaN        None         None         None   
2Wire                    None        None         None  $53,500,000   
3-V Biosciences          None        None  $30,000,000  $20,000,000   
37coins              $500,000        None         None         None   
3Jam                     None  $4,000,000         None         None   
3LM                $1,500,000        None         None         None   
3TEN8.AI           $1,670,000        None         None         None   
3VR                      None        None  $10,000,000   $9,000,000   
41st Parameter           None        None  $11,200,000  $10,070,000   
42 Technologies           NaN        None         None         None   

Funding Type          Seed_date        A_date        B_date        C_date  \
Organization Name                                                           
1Huddle            Oct 20, 2015          None          None          None   
1mg                        None  Apr 20, 2015  Apr 20, 2016  Jul 26, 2017   
1stdibs             Oct 1, 2012          None          None          None   
20n                 Dec 1, 2014          None          None          None   
20x200                     None   Oct 1, 2009          None          None   
2345.com                   None          None   Oct 1, 2010          None   
23andMe                    None   Oct 3, 2007  Jun 18, 2009   Nov 9, 2010   
247tickets         Aug 22, 2016          None          None          None   
2Wire                      None          None          None  Apr 24, 2000   
3-V Biosciences            None          None  Jul 10, 2009  Jun 21, 2013   
37coins             Jun 1, 2014          None          None          None   
3Jam                       None   Jul 1, 2007          None          None   
3LM                 Jul 1, 2010          None          None          None   
3TEN8.AI           Apr 15, 2015          None          None          None   
3VR                        None          None  Jan 17, 2006  Sep 28, 2006   
41st Parameter             None          None   May 8, 2006   Aug 9, 2007   
42 Technologies    May 16, 2014          None          None          None   

Funding Type                 index  
Organization Name                   
1Huddle                    1Huddle  
1mg                            1mg  
1stdibs                    1stdibs  
20n                            20n  
20x200                      20x200  
2345.com                  2345.com  
23andMe                    23andMe  
247tickets              247tickets  
2Wire                        2Wire  
3-V Biosciences    3-V Biosciences  
37coins                    37coins  
3Jam                          3Jam  
3LM                            3LM  
3TEN8.AI                  3TEN8.AI  
3VR                            3VR  
41st Parameter      41st Parameter  
42 Technologies    42 Technologies

In [288]:
amount_list = ["Seed", "Series A", "Series B", "Series C"]

def fill_na(amount_list, df):
    for fund in amount_list:
        df[fund] = df[fund].fillna("$0")

In [291]:
fill_na(amount_list, f_trans_all)
f_trans_all

Seed     Series A     Series B  \
Organization Name                                                              
 Justmop.com                                    $0           $0           $0   
 NimbleRx                                       $0  $12,000,000  $28,000,000   
 SpotDraft                                $550,000           $0           $0   
/dev/color                                $120,000           $0           $0   
10 By 10                                  $120,000           $0           $0   
1000memories                                    $0   $2,520,000           $0   
1006.tv                                         $0           $0  $10,000,000   
140 Proof                                       $0           $0   $2,500,000   
1Huddle                                   $125,000           $0           $0   
1mg                                             $0   $6,000,000  $16,000,000   
1stdibs                                         $0           $0           $0   
20n                                       $120,000           $0           $0   
20x200                                          $0     $800,000           $0   
2345.com                                        $0           $0           $0   
23andMe                                         $0   $8,953,320  $13,600,000   
247tickets                                      $0           $0           $0   
280 North                                       $0           $0           $0   
2Wire                                           $0           $0           $0   
3-V Biosciences                                 $0           $0  $30,000,000   
37coins                                   $500,000           $0           $0   
3Jam                                            $0   $4,000,000           $0   
3LM                                     $1,500,000           $0           $0   
3TEN8.AI                                $1,670,000           $0           $0   
3VR                                             $0           $0  $10,000,000   
3ware                                           $0           $0           $0   
41st Parameter                                  $0           $0  $11,200,000   
42 Technologies                                 $0           $0           $0   
42Floors                                  $400,000           $0  $12,000,000   
43Layers                                        $0           $0           $0   
4INFO                                           $0           $0           $0   
...                                            ...          ...          ...   
payleven                                        $0   €2,000,000  €12,500,000   
play140                                         $0           $0           $0   
quixi                                           $0  $27,500,000           $0   
rakam                                     $150,000           $0           $0   
reMail                                          $0           $0           $0   
resin.io                                        $0   $3,000,000           $0   
seeUthere.com                                   $0           $0           $0   
simplifund                                $125,000           $0           $0   
slinkset                                        $0           $0           $0   
so-sure                                   $150,000           $0           $0   
sourceeasy                                $126,500           $0           $0   
sp0n                                            $0  $12,000,000           $0   
springcleaning.ae                               $0           $0           $0   
storefront                                      $0   $7,300,000           $0   
strapping                                 $100,000           $0           $0   
tCell                                           $0   $9,400,000           $0   
tagMonkey                                 $125,000           $0           $0   
terraspring                                   

In [300]:
funds = ["Seed", "Series A", "Series B", "Series C"]
currency_col = ["S_c", "A_c", "B_c", "C_c"]


def fund_currency(funds, df):
    for fund in funds:
        new_col_name = ""
        if fund == "Seed":
            new_col_name = fund[0] + "_c"
        else:
            new_col_name = fund[-1] + "_c"
        df[new_col_name] = df[fund].str.replace("[0-9]|[,]", "")



def fund_to_int(funds, df):
    for fund in funds:
        new_col_name = ""
        if fund == "Seed":
            new_col_name = fund[0] + "_int"
        else:
            new_col_name = fund[-1] + "_int"
        df[new_col_name] = df[fund].str.replace("[^0-9]+|[,]", "")

        
def currencies(currency_col, df):
    list_of_currencies = []
    
    for currencies in currency_col:
        for currency in df[currencies].unique():
            list_of_currencies.append(currency)
    return list_of_currencies

In [301]:
fund_currency(funds, f_trans_all)
fund_to_int(funds, f_trans_all)

In [303]:
curr = currencies(currency_col, f_trans_all)
curr

['$',
 '£',
 '₩',
 '¥',
 'A$',
 'CA$',
 'R$',
 '€',
 'SGD',
 'MYR',
 '₹',
 '$',
 '£',
 'CN¥',
 '€',
 'CA$',
 '₹',
 'SGD',
 'A$',
 '¥',
 '$',
 'CN¥',
 '£',
 '€',
 '₹',
 '$',
 '€',
 '₹',
 'A$',
 'CN¥',
 '£']

In [294]:
f_trans_all.columns

Index(['Seed', 'Series A', 'Series B', 'Series C', 'Seed_date', 'A_date',
       'B_date', 'C_date', 'index', 'Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount', 'Seed_date_dt',
       'A_date_dt', 'Seed_A', 'B_date_dt', 'C_date_dt', 'S_to_A', 'A_to_B',
       'B_to_C', 'S_c', 'A_c', 'B_c', 'C_c', 'S_int', 'A_int', 'B_int',
       'C_int'],
      dtype='object')

In [306]:
f_trans_all.head(10).iloc[:,16:]

Seed_date_dt  A_date_dt  Seed_A  B_date_dt  C_date_dt  \
Organization Name                                                         
 Justmop.com               NaT 2017-11-21     NaT        NaT        NaT   
 NimbleRx                  NaT 2015-10-13     NaT 2017-10-24        NaT   
 SpotDraft          2017-10-25        NaT     NaT        NaT        NaT   
/dev/color          2016-08-23        NaT     NaT        NaT        NaT   
10 By 10            2017-07-01        NaT     NaT        NaT        NaT   
1000memories               NaT 2011-02-16     NaT        NaT        NaT   
1006.tv                    NaT        NaT     NaT 2014-07-31        NaT   
140 Proof                  NaT        NaT     NaT 2011-04-28        NaT   
1Huddle             2015-10-20        NaT     NaT        NaT        NaT   
1mg                        NaT 2015-04-20     NaT 2016-04-20 2017-07-26   

                   S_to_A   A_to_B   B_to_C S_c A_c B_c C_c   S_int     A_int  \
Organization Name                                                               
 Justmop.com          NaT      NaT      NaT   $   $   $   $       0         0   
 NimbleRx             NaT 742 days      NaT   $   $   $   $       0  12000000   
 SpotDraft            NaT      NaT      NaT   $   $   $   $  550000         0   
/dev/color            NaT      NaT      NaT   $   $   $   $  120000         0   
10 By 10              NaT      NaT      NaT   $   $   $   $  120000         0   
1000memories          NaT      NaT      NaT   $   $   $   $       0   2520000   
1006.tv               NaT      NaT      NaT   $   $   $   $       0         0   
140 Proof             NaT      NaT      NaT   $   $   $   $       0         0   
1Huddle               NaT      NaT      NaT   $   $   $   $  125000         0   
1mg                   NaT 366 days 462 days   $   $   $   $       0   6000000   

                      B_int     C_int  
Organization Name                      
 Justmop.com              0         0  
 NimbleRx          28000000         0  
 SpotDraft                0         0  
/dev/color                0         0  
10 By 10                  0         0  
1000memories              0         0  
1006.tv            10000000         0  
140 Proof           2500000         0  
1Huddle                   0         0  
1mg                16000000  15000000

In [265]:
f_trans_all.drop_duplicates(subset = ["Seed_c", "A_c", "B_c", "C_c"], )[["Seed_c", "A_c", "B_c", "C_c"]]

Seed_c  A_c  B_c  C_c
Organization Name                               
 Justmop.com                    $    $    $    $
Airsorted                       £    $    $    $
Althea Inc.                     ₩    $    $    $
Angel Medical Group             $    $  CN¥    $
Aveillant                       $    £    £    $
Baidu Literature                $  CN¥    $    $
Bizimply                        $    €    $    $
Bus.com                         $  CA$    $    $
Campanda                        $    €    €    $
Carwow                          $    $    £    $
Catawiki                        $    $    €    $
Collectabillia.com              $    ₹    ₹    $
Credit Engine                   ¥    $    $    $
Dailyhunt                       $    $    ₹    $
FIRE1                           $    $    $    €
Five Star Business Finance      $    $    $    ₹
FreeCharge                      $    ₹    $    $
Funding Societies               $  SGD    $    $
GO1                            A$    $    $    $
Gilmour Space Technologies      $   A$    $    $
Gymtrack                      CA$    $    $    $
HealthEngine                    $    $    $   A$
Infervision                     $  CN¥  CN¥    $
Kuaikan Comic                   $    $    $  CN¥
Magnetis                       R$    $    $    $
Median Technologies             $    $    €    €
Metabolomic Diagnostics         €    $    $    $
MetroResidences               SGD    $    $    $
Mobingi. Inc.                   $    ¥    $    $
Naluri Hidup                  MYR    $    $    $
Omni                            $    £    $    $
Sheroes                         ₹    $    $    $
Wonga                           $    $    $    £

In [257]:
list(set(curr))

['CN¥', 'CA$', '¥', '₹', 'A$', '€', 'MYR', 'SGD', '$', '₩', '£', 'R$']

In [239]:
# f_trans_all["Seed_int"] = f_trans_all["Seed"].str.replace("[^0-9]|[,]", "")
# f_trans_all["A_int"] = f_trans_all["Series A"].str.replace("[^0-9]+|[,]", "")
# f_trans_all["B_int"] = f_trans_all["Series B"].str.replace("[^0-9]+|[,]", "")
# f_trans_all["C_int"] = f_trans_all["Series C"].str.replace("[^0-9]+|[,]", "")
# f_trans_all

In [240]:
# f_trans_all["Seed_c"] = f_trans_all["Seed"].str.replace("[0-9]+|[,]", "")
# f_trans_all["A_c"] = f_trans_all["Series A"].str.replace("[0-9]+|[,]", "")
# f_trans_all["B_c"] = f_trans_all["Series B"].str.replace("[0-9]+|[,]", "")
# f_trans_all["C_c"] = f_trans_all["Series C"].str.replace("[0-9]+|[,]", "")
# f_trans_all[f_trans_all.Seed_c.str.contains('£|₩|¥|A$|CA$|R$')][["Seed_c", "A_c", "B_c", "C_c"]]

In [218]:
currency = list(f_trans_all["Seed"].str.replace("[0-9]+|[,]", "").unique())
currency

['$', '£', '₩', '¥', 'A$', 'CA$', 'R$', '€', 'SGD', 'MYR', '₹']

In [221]:
currency.append(list(f_trans_all["Series C"].str.replace("[0-9]+|[,]", "").unique()))

currency

['$',
 '£',
 '₩',
 '¥',
 'A$',
 'CA$',
 'R$',
 '€',
 'SGD',
 'MYR',
 '₹',
 ['$', '€', '₹', 'A$', 'CN¥', '£']]

In [202]:
f_trans_all["Seed_currency"] = f_trans_all["Seed"].str.replace("[0-9]+|[,]", "")

In [204]:
f_trans_all["Seed_currency"].unique()

array(['$', '£', '₩', '¥', 'A$', 'CA$', 'R$', '€', 'SGD', 'MYR', '₹'], dtype=object)

In [180]:
currency = re.compile(r'^[A-z]?[$£₩¥€]')

In [207]:
# currency_list = []

# for money in f_trans_all["Seed"]:
#     amount = re.compile(r'^[A-z]?[$£₩¥€]')
# #     if money :
# #         print(money)
#     currency = amount.search(money)
#     if currency is not None:
#         currency_list.append(currency.group())
    
# f_trans_all["Seed_currency"] = currency_list

ValueError: Length of values does not match length of index

In [196]:
%time list(set([1, 1.5, 5, 0.8, 100, 50, 0.78]))

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 14.1 µs


[0.8, 1, 1.5, 0.78, 100, 5, 50]

In [192]:
rate = set([1, 1.5, 5, 0.8, 100, 50, 0.78])

In [193]:
curr = set(currency_list)

In [194]:
curr
rate

{0.78, 0.8, 1, 1.5, 5, 50, 100}

In [138]:
f_trans_all.dtypes

Seed                                    object
Series A                                object
Series B                                object
Series C                                object
Seed_date                               object
A_date                                  object
B_date                                  object
C_date                                  object
index                                   object
Categories                              object
Category Groups                         object
Founded Date                            object
Funding Status                          object
Operating Status                        object
Total Funding Amount                    object
Total Equity Funding Amount             object
Seed_date_dt                    datetime64[ns]
A_date_dt                       datetime64[ns]
Seed_A                         timedelta64[ns]
B_date_dt                       datetime64[ns]
C_date_dt                       datetime64[ns]
S_to_A       

In [132]:
f_trans_all["Seed_int"] = f_trans_all["Seed"].str.replace("^[A-z]?[$£₩¥€]", "")
f_trans_all["Seed_int"] = f_trans_all["Seed_int"].str.replace(",", "")

In [133]:
f_trans_all[["Seed_int", "Seed"]]

Seed_int        Seed
Organization Name                                          
 Justmop.com                               None        None
 NimbleRx                                  None        None
 SpotDraft                               550000    $550,000
/dev/color                               120000    $120,000
10 By 10                                 120000    $120,000
1000memories                               None        None
1006.tv                                    None        None
140 Proof                                  None        None
1Huddle                                  125000    $125,000
1mg                                        None        None
1stdibs                                     NaN         NaN
20n                                      120000    $120,000
20x200                                     None        None
2345.com                                   None        None
23andMe                                    None        None
247tickets                                  NaN         NaN
280 North                                   NaN         NaN
2Wire                                      None        None
3-V Biosciences                            None        None
37coins                                  500000    $500,000
3Jam                                       None        None
3LM                                     1500000  $1,500,000
3TEN8.AI                                1670000  $1,670,000
3VR                                        None        None
3ware                                       NaN         NaN
41st Parameter                             None        None
42 Technologies                             NaN         NaN
42Floors                                 400000    $400,000
43Layers                                    NaN         NaN
4INFO                                      None        None
...                                         ...         ...
payleven                                   None        None
play140                                     NaN         NaN
quixi                                      None        None
rakam                                    150000    $150,000
reMail                                      NaN         NaN
resin.io                                   None        None
seeUthere.com                               NaN         NaN
simplifund                               125000    $125,000
slinkset                                    NaN         NaN
so-sure                                  150000    $150,000
sourceeasy                               126500    $126,500
sp0n                                       None        None
springcleaning.ae                          None        None
storefront                                 None        None
strapping                                100000    $100,000
tCell                                      None        None
tagMonkey                                125000    $125,000
terraspring                                 NaN         NaN
textPlus                                   None        None
uBiome                                   120000    $120,000
uShip                                      None        None
unspun, inc.                                NaN         NaN
userfox                                     NaN         NaN
vLine                                   1500000  $1,500,000
videogram                                  None        None
vip.com                                    None        None
wedgies                                  550000    $550,000
wifi.com.ng                             2100000  $2,100,000
xendit                                   120000    $120,000
zPREDICTA (formerly Ixchel Scientific)   120000    $120,000

[5006 rows x 2 columns]

In [119]:
f_trans_all[f_trans_all["Seed_int"].str.contains("^[A-z]?[$£₩¥€]") == True]

Empty DataFrame
Columns: [Seed, Series A, Series B, Series C, Seed_date, A_date, B_date, C_date, index, Categories, Category Groups, Founded Date, Funding Status, Operating Status, Total Funding Amount, Total Equity Funding Amount, Seed_date_dt, A_date_dt, Seed_A, B_date_dt, C_date_dt, S_to_A, A_to_B, B_to_C, Seed_int]
Index: []

[0 rows x 25 columns]

In [308]:
f_trans_all.to_csv("../output/f_trans_all.csv")

In [31]:
# pd.to_datetime(f_trans_all, "A_date", format="%b %d, %Y", errors = 'coerce')

TypeError: to_datetime() got multiple values for argument 'errors'

In [32]:
# currency_to_int(f_trans_all, "Seed", "Seed_int")

ValueError: could not convert string to float: '₹54000000'

In [ ]:
# f_trans_all.datetime

In [ ]:
# f_trans_all[["Seed", "Seed_int"]]

In [ ]:
# f_trans_all[f_trans_all["Seed_int"] == "$1000000"]

In [ ]:
# f_trans_all["Series B"].str.replace(",", "")

## Duplicates are predominantly null values.

In [ ]:
print(f_trans_all[f_trans_all.duplicated(keep = False)].shape)
f_trans_all[f_trans_all.duplicated(keep = False)]